In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt
import multiprocessing as mp

/home/ruben/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (8,8)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    rows,cols = dimensions
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    for index, x in np.random.permutation(list(np.ndenumerate(board))):
        row,col = index
        if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
            clearSquare(board,result,row,col)
    return result

In [5]:
# Generates a random training batch of size n
def randomBoard(i):
    return(np.random.random(dimensions) < mineProbability)

def encodeCountsOneHot(counts):
    countsOneHot = np.zeros((counts.size,10))
    countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
    return(countsOneHot.flatten())

def validGuesses(boardAndCounts):
    board,counts = boardAndCounts
    validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
        board.flatten().astype(float))
    validGuessesSum = sum(validGuesses)
    if validGuessesSum > 0:
        return(validGuesses / validGuessesSum)
    else:
        return(np.zeros(board.size*2))

try:
    cpus = mp.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default

pool = mp.Pool(processes=cpus)

def next_training_batch(n):
    boards = pool.map(randomBoard, range(n))
    counts = pool.map(boardPartialMineCounts, boards)
    batch_xs = pool.map(encodeCountsOneHot, counts)
    batch_ys = pool.map(validGuesses, zip(boards,counts))
    return(batch_xs, batch_ys, boards)

In [6]:
rows, cols = dimensions
size = rows*cols

In [7]:
model = tf.keras.models.Sequential()

In [8]:
model.add(tf.keras.layers.Dense(units=size, input_dim=size*10))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Reshape((rows, cols, 1)))

model.add(tf.keras.layers.Conv2D(32, (5, 5), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Reshape((4*4*32,)))

#model.add(tf.keras.layers.Dense(units=1024))
#model.add(tf.keras.layers.Activation('relu'))

#model.add(tf.keras.layers.Dropout(rate=0.5))

model.add(tf.keras.layers.Dense(units=size*2))
model.add(tf.keras.layers.Activation('softmax'))

In [9]:
model.compile(optimizer='adam',
              loss='poisson',
              metrics=['accuracy'])

In [10]:
print(model.metrics_names)

['loss', 'acc']


In [11]:
savePath = '/media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines8'

In [11]:
for iteration in range(1000001):
    x_batch, y_batch, _ = next_training_batch(10000)
    loss, accuracy = model.train_on_batch(np.array(x_batch), np.array(y_batch))
    print('%s: Loss and accuracy at step %s: %s, %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss, accuracy))
    if iteration % 500 == 0:
        model.save('%s/model-%s.hs' % savePath, iteration)

2017-11-10 14:30:22: Loss and accuracy at step 0: 0.0457546, 0.004
2017-11-10 14:30:24: Loss and accuracy at step 1: 0.0457211, 0.0307
2017-11-10 14:30:26: Loss and accuracy at step 2: 0.0456961, 0.0994
2017-11-10 14:30:28: Loss and accuracy at step 3: 0.0456614, 0.1845
2017-11-10 14:30:31: Loss and accuracy at step 4: 0.0456281, 0.2248
2017-11-10 14:30:33: Loss and accuracy at step 5: 0.0455972, 0.2615
2017-11-10 14:30:35: Loss and accuracy at step 6: 0.0455677, 0.2972
2017-11-10 14:30:37: Loss and accuracy at step 7: 0.0455334, 0.3123
2017-11-10 14:30:39: Loss and accuracy at step 8: 0.0455119, 0.3297
2017-11-10 14:30:41: Loss and accuracy at step 9: 0.0454891, 0.3391
2017-11-10 14:30:43: Loss and accuracy at step 10: 0.0454549, 0.3418
2017-11-10 14:30:45: Loss and accuracy at step 11: 0.0454475, 0.3338
2017-11-10 14:30:47: Loss and accuracy at step 12: 0.0454402, 0.3356
2017-11-10 14:30:49: Loss and accuracy at step 13: 0.0454124, 0.3241
2017-11-10 14:30:51: Loss and accuracy at ste

2017-11-10 14:34:31: Loss and accuracy at step 120: 0.0394767, 0.0315
2017-11-10 14:34:33: Loss and accuracy at step 121: 0.0394613, 0.0235
2017-11-10 14:34:35: Loss and accuracy at step 122: 0.0394703, 0.0229
2017-11-10 14:34:37: Loss and accuracy at step 123: 0.0394517, 0.0328
2017-11-10 14:34:39: Loss and accuracy at step 124: 0.0394302, 0.0283
2017-11-10 14:34:41: Loss and accuracy at step 125: 0.0393682, 0.0206
2017-11-10 14:34:43: Loss and accuracy at step 126: 0.0393593, 0.0264
2017-11-10 14:34:45: Loss and accuracy at step 127: 0.0393721, 0.0347
2017-11-10 14:34:47: Loss and accuracy at step 128: 0.0392923, 0.0223
2017-11-10 14:34:49: Loss and accuracy at step 129: 0.0392085, 0.0277
2017-11-10 14:34:51: Loss and accuracy at step 130: 0.0393568, 0.0348
2017-11-10 14:34:53: Loss and accuracy at step 131: 0.0393104, 0.0259
2017-11-10 14:34:55: Loss and accuracy at step 132: 0.0392658, 0.0206
2017-11-10 14:34:57: Loss and accuracy at step 133: 0.0392168, 0.034
2017-11-10 14:34:59: 

2017-11-10 14:38:36: Loss and accuracy at step 238: 0.0381651, 0.0218
2017-11-10 14:38:38: Loss and accuracy at step 239: 0.0380896, 0.0158
2017-11-10 14:38:40: Loss and accuracy at step 240: 0.0380644, 0.0285
2017-11-10 14:38:42: Loss and accuracy at step 241: 0.0380941, 0.0183
2017-11-10 14:38:44: Loss and accuracy at step 242: 0.038111, 0.0243
2017-11-10 14:38:46: Loss and accuracy at step 243: 0.0381304, 0.0209
2017-11-10 14:38:48: Loss and accuracy at step 244: 0.0380736, 0.0203
2017-11-10 14:38:50: Loss and accuracy at step 245: 0.0379929, 0.0271
2017-11-10 14:38:53: Loss and accuracy at step 246: 0.0380739, 0.0164
2017-11-10 14:38:55: Loss and accuracy at step 247: 0.0380343, 0.0167
2017-11-10 14:38:57: Loss and accuracy at step 248: 0.0380299, 0.029
2017-11-10 14:38:59: Loss and accuracy at step 249: 0.0380985, 0.0179
2017-11-10 14:39:01: Loss and accuracy at step 250: 0.0380822, 0.0174
2017-11-10 14:39:03: Loss and accuracy at step 251: 0.0380748, 0.0303
2017-11-10 14:39:05: L

2017-11-10 14:42:42: Loss and accuracy at step 356: 0.0376803, 0.021
2017-11-10 14:42:44: Loss and accuracy at step 357: 0.0377147, 0.0223
2017-11-10 14:42:46: Loss and accuracy at step 358: 0.0376877, 0.0132
2017-11-10 14:42:48: Loss and accuracy at step 359: 0.0377075, 0.024
2017-11-10 14:42:51: Loss and accuracy at step 360: 0.0377018, 0.0195
2017-11-10 14:42:53: Loss and accuracy at step 361: 0.0375855, 0.014
2017-11-10 14:42:55: Loss and accuracy at step 362: 0.0377411, 0.0251
2017-11-10 14:42:57: Loss and accuracy at step 363: 0.0377246, 0.0164
2017-11-10 14:42:59: Loss and accuracy at step 364: 0.0376231, 0.0165
2017-11-10 14:43:01: Loss and accuracy at step 365: 0.0375867, 0.0127
2017-11-10 14:43:03: Loss and accuracy at step 366: 0.0376023, 0.0246
2017-11-10 14:43:05: Loss and accuracy at step 367: 0.0376761, 0.0243
2017-11-10 14:43:07: Loss and accuracy at step 368: 0.0377555, 0.0107
2017-11-10 14:43:09: Loss and accuracy at step 369: 0.0377771, 0.0266
2017-11-10 14:43:11: Lo

2017-11-10 14:46:48: Loss and accuracy at step 474: 0.0374174, 0.0183
2017-11-10 14:46:50: Loss and accuracy at step 475: 0.0374606, 0.0243
2017-11-10 14:46:53: Loss and accuracy at step 476: 0.0374728, 0.0106
2017-11-10 14:46:55: Loss and accuracy at step 477: 0.037494, 0.0221
2017-11-10 14:46:57: Loss and accuracy at step 478: 0.0374981, 0.0232
2017-11-10 14:46:59: Loss and accuracy at step 479: 0.0375274, 0.0131
2017-11-10 14:47:01: Loss and accuracy at step 480: 0.0375019, 0.0196
2017-11-10 14:47:03: Loss and accuracy at step 481: 0.0375662, 0.0217
2017-11-10 14:47:05: Loss and accuracy at step 482: 0.0375107, 0.0157
2017-11-10 14:47:07: Loss and accuracy at step 483: 0.0374838, 0.0116
2017-11-10 14:47:09: Loss and accuracy at step 484: 0.037439, 0.024
2017-11-10 14:47:11: Loss and accuracy at step 485: 0.0373736, 0.0202
2017-11-10 14:47:13: Loss and accuracy at step 486: 0.0374414, 0.0138
2017-11-10 14:47:16: Loss and accuracy at step 487: 0.0373282, 0.0175
2017-11-10 14:47:18: Lo

...

2017-11-10 18:09:10: Loss and accuracy at step 6283: 0.0367461, 0.0162
2017-11-10 18:09:12: Loss and accuracy at step 6284: 0.0367209, 0.0085
2017-11-10 18:09:14: Loss and accuracy at step 6285: 0.0367322, 0.0146
2017-11-10 18:09:16: Loss and accuracy at step 6286: 0.0367618, 0.013
2017-11-10 18:09:18: Loss and accuracy at step 6287: 0.0367769, 0.0139
2017-11-10 18:09:20: Loss and accuracy at step 6288: 0.0368193, 0.0137
2017-11-10 18:09:22: Loss and accuracy at step 6289: 0.0367664, 0.0127
2017-11-10 18:09:25: Loss and accuracy at step 6290: 0.036823, 0.0129
2017-11-10 18:09:27: Loss and accuracy at step 6291: 0.0367497, 0.0136
2017-11-10 18:09:29: Loss and accuracy at step 6292: 0.0366968, 0.0115
2017-11-10 18:09:31: Loss and accuracy at step 6293: 0.0367314, 0.0126
2017-11-10 18:09:33: Loss and accuracy at step 6294: 0.0367448, 0.0128
2017-11-10 18:09:35: Loss and accuracy at step 6295: 0.0368266, 0.0136
2017-11-10 18:09:37: Loss and accuracy at step 6296: 0.036777, 0.0093
2017-11-1

2017-11-10 18:13:13: Loss and accuracy at step 6399: 0.0367843, 0.0169
2017-11-10 18:13:15: Loss and accuracy at step 6400: 0.0367233, 0.0124
2017-11-10 18:13:17: Loss and accuracy at step 6401: 0.0368223, 0.0119
2017-11-10 18:13:19: Loss and accuracy at step 6402: 0.0367482, 0.018
2017-11-10 18:13:22: Loss and accuracy at step 6403: 0.0366251, 0.0117
2017-11-10 18:13:24: Loss and accuracy at step 6404: 0.036715, 0.0119
2017-11-10 18:13:26: Loss and accuracy at step 6405: 0.036728, 0.014
2017-11-10 18:13:28: Loss and accuracy at step 6406: 0.0366592, 0.0068
2017-11-10 18:13:30: Loss and accuracy at step 6407: 0.0367643, 0.0185
2017-11-10 18:13:32: Loss and accuracy at step 6408: 0.0367649, 0.0114
2017-11-10 18:13:34: Loss and accuracy at step 6409: 0.0366536, 0.0131
2017-11-10 18:13:36: Loss and accuracy at step 6410: 0.0367691, 0.0149
2017-11-10 18:13:38: Loss and accuracy at step 6411: 0.0367679, 0.0107
2017-11-10 18:13:40: Loss and accuracy at step 6412: 0.036766, 0.0153
2017-11-10 

2017-11-10 18:17:16: Loss and accuracy at step 6515: 0.0367036, 0.0188
2017-11-10 18:17:18: Loss and accuracy at step 6516: 0.0367586, 0.0095
2017-11-10 18:17:20: Loss and accuracy at step 6517: 0.0366698, 0.0103
2017-11-10 18:17:22: Loss and accuracy at step 6518: 0.0368137, 0.0169
2017-11-10 18:17:24: Loss and accuracy at step 6519: 0.0366605, 0.0091
2017-11-10 18:17:26: Loss and accuracy at step 6520: 0.0368488, 0.0196
2017-11-10 18:17:28: Loss and accuracy at step 6521: 0.0368663, 0.0089
2017-11-10 18:17:31: Loss and accuracy at step 6522: 0.0368532, 0.0153
2017-11-10 18:17:33: Loss and accuracy at step 6523: 0.0368568, 0.0107
2017-11-10 18:17:35: Loss and accuracy at step 6524: 0.0368015, 0.0095
2017-11-10 18:17:37: Loss and accuracy at step 6525: 0.0367092, 0.0183
2017-11-10 18:17:39: Loss and accuracy at step 6526: 0.0367529, 0.0066
2017-11-10 18:17:41: Loss and accuracy at step 6527: 0.0368319, 0.0215
2017-11-10 18:17:43: Loss and accuracy at step 6528: 0.0367498, 0.0127
2017-1

2017-11-10 18:21:22: Loss and accuracy at step 6631: 0.0368122, 0.0081
2017-11-10 18:21:24: Loss and accuracy at step 6632: 0.0367666, 0.0149
2017-11-10 18:21:26: Loss and accuracy at step 6633: 0.0367654, 0.0141
2017-11-10 18:21:29: Loss and accuracy at step 6634: 0.0367928, 0.0091
2017-11-10 18:21:31: Loss and accuracy at step 6635: 0.0367243, 0.0189
2017-11-10 18:21:33: Loss and accuracy at step 6636: 0.0367024, 0.012
2017-11-10 18:21:35: Loss and accuracy at step 6637: 0.0367732, 0.0112
2017-11-10 18:21:37: Loss and accuracy at step 6638: 0.0368069, 0.0259
2017-11-10 18:21:39: Loss and accuracy at step 6639: 0.0368623, 0.0098
2017-11-10 18:21:41: Loss and accuracy at step 6640: 0.0367947, 0.0113
2017-11-10 18:21:43: Loss and accuracy at step 6641: 0.0367505, 0.0271
2017-11-10 18:21:45: Loss and accuracy at step 6642: 0.0368182, 0.0123
2017-11-10 18:21:48: Loss and accuracy at step 6643: 0.0368639, 0.0107
2017-11-10 18:21:50: Loss and accuracy at step 6644: 0.0367635, 0.0173
2017-11

KeyboardInterrupt: 